<a href="https://colab.research.google.com/github/SG7504/GFGKIIT-Wanderers-Hyperthon/blob/main/HyperThonFINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pydub
!pip install torch torchvision torchaudio
!pip install transformers
!pip install openai-whisper
!pip install webrtcvad
!pip install noisereduce
!pip install srt
!pip install librosa
!apt-get install ffmpeg
!pip install scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 42.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 65.9 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803373 sha256=e2f24a3fe41da3ddb907c84d6aabe9da77d748c98e158252d17f852f4480e0fe
  Stored in directory: /root/.cache/pip/wheels/2f/f2/ce/6eb23db4091d026238ce76703bd66da60b969d70bcc81d5d3a
Successfully built openai-whisper
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for webrtcvad: filename=webrtcvad-2.0.10-cp311-cp311-linux_x86_64.whl size=73498 sha256=7bb4084f93436a6bad394781eddf516ae563acef4d102d06ad8921051ba60b49
  Stored in directory: /root/.cache/pip/wheels/94/65/3f/292d0b656be33d1c801831201c74b5f68f41a2ae465ff2ee

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
from pydub import AudioSegment
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
import whisper
import srt
from datetime import timedelta
import webrtcvad
import wave

def preprocess_audio_for_vad(input_audio_path, output_audio_path):
    """
    Preprocess audio for WebRTC VAD:
    - Convert to mono, 16-bit PCM, 16 kHz WAV format
    - Extract speech using WebRTC VAD
    """
    print("Preprocessing audio for VAD...")

    # Load and convert audio to mono and 16 kHz
    audio = AudioSegment.from_file(input_audio_path)
    audio = audio.set_channels(1)  # Mono
    audio = audio.set_frame_rate(16000)  # 16 kHz

    # Export audio to a temporary WAV file
    temp_wav_path = "temp_audio.wav"
    audio.export(temp_wav_path, format="wav")

    # Read the WAV file and process it with WebRTC VAD
    vad = webrtcvad.Vad()
    vad.set_mode(2)  # Balanced mode for speech detection

    with wave.open(temp_wav_path, "rb") as wf:
        # Check audio format
        assert wf.getnchannels() == 1, "Audio must be mono"
        assert wf.getsampwidth() == 2, "Audio must be 16-bit PCM"
        assert wf.getframerate() == 16000, "Audio must have 16 kHz sample rate"

        frame_duration = 30  # ms (10, 20, or 30 ms frames supported)
        frame_size = int(wf.getframerate() * (frame_duration / 1000)) * wf.getsampwidth()

        speech_frames = []
        while True:
            frames = wf.readframes(frame_size // wf.getsampwidth())
            if len(frames) < frame_size:
                break  # End of file

            # Use WebRTC VAD to detect speech
            is_speech = vad.is_speech(frames, wf.getframerate())
            if is_speech:
                speech_frames.append(frames)

    # Combine speech frames into a new audio file
    with wave.open(output_audio_path, "wb") as out_wf:
        out_wf.setnchannels(1)
        out_wf.setsampwidth(2)
        out_wf.setframerate(16000)
        out_wf.writeframes(b"".join(speech_frames))

    print(f"Preprocessed audio saved to: {output_audio_path}")


# Transcribe audio to text using Whisper
def transcribe_audio_whisper(audio_path, whisper_model):
    print("Transcribing audio...")
    result = whisper_model.transcribe(audio_path, language="ja")
    print("Transcription completed.")
    return result["segments"]


# Translate text using M2M-100
def translate_text_m2m100(text, m2m_model, m2m_tokenizer, src_lang="ja", tgt_lang="en"):
    m2m_tokenizer.src_lang = src_lang
    encoded = m2m_tokenizer(text, return_tensors="pt")
    generated_tokens = m2m_model.generate(
        **encoded,
        forced_bos_token_id=m2m_tokenizer.lang_code_to_id[tgt_lang],
        max_length=512
    )
    return m2m_tokenizer.decode(generated_tokens[0], skip_special_tokens=True)


# Create an SRT file from transcription and translation
def create_srt(transcriptions, model, tokenizer):
    subtitles = []
    for segment in transcriptions:
        start = timedelta(seconds=segment["start"])
        end = timedelta(seconds=segment["end"])
        japanese_text = segment["text"]
        english_translation = translate_text_m2m100(japanese_text, model, tokenizer)

        subtitle = srt.Subtitle(
            index=len(subtitles) + 1,
            start=start,
            end=end,
            content=f"{english_translation}\n({japanese_text})",
        )
        subtitles.append(subtitle)
    return subtitles


# Main function
def main(input_video_path, output_srt_path, temp_audio_path="temp_audio.wav"):
    # Load Whisper and M2M-100 models
    print("Loading models...")
    whisper_model = whisper.load_model("large")  # Use Whisper large model for better accuracy
    m2m_model_name = "facebook/m2m100_418M"
    m2m_tokenizer = M2M100Tokenizer.from_pretrained(m2m_model_name)
    m2m_model = M2M100ForConditionalGeneration.from_pretrained(m2m_model_name)
    print("Models loaded successfully.")

    # Preprocess audio
    preprocess_audio_for_vad(input_video_path, temp_audio_path)

    # Transcribe audio
    transcriptions = transcribe_audio_whisper(temp_audio_path, whisper_model)

    # Create subtitles
    print("Creating SRT file...")
    subtitles = create_srt(transcriptions, m2m_model, m2m_tokenizer)
    with open(output_srt_path, "w", encoding="utf-8") as f:
        f.write(srt.compose(subtitles))
    print(f"Subtitle file created: {output_srt_path}")

    # Cleanup temporary files
    if os.path.exists(temp_audio_path):
        os.remove(temp_audio_path)


# Run the pipeline
if __name__ == "__main__":
    VIDEO_PATH = "/content/drive/MyDrive/Hyperthon/test.mp4"  # Replace with your actual file path
    OUTPUT_SRT_PATH = "/content/drive/MyDrive/Hyperthon/outputNew2.srt"  # Replace with the desired output path

    main(VIDEO_PATH, OUTPUT_SRT_PATH)


Loading models...


100%|█████████████████████████████████████| 2.88G/2.88G [00:36<00:00, 84.3MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

tokenizer_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

Models loaded successfully.
Preprocessing audio for VAD...
Preprocessed audio saved to: temp_audio.wav
Transcribing audio...
Transcription completed.
Creating SRT file...
Subtitle file created: /content/drive/MyDrive/Hyperthon/outputNew2.srt
